<a href="https://colab.research.google.com/github/swarooprm/CSE-576-NR-2/blob/master/NLPProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import project library from github:

In [4]:
!git clone https://github.com/ag1988/injecting_numeracy.git

Cloning into 'injecting_numeracy'...


Checking the current working directory:

In [2]:
# !pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


Navigate to the pre-training directory:

In [5]:
%cd injecting_numeracy/pre_training/

D:\Graduation Courses\Semester 2\NLP\Project\project_git\CSE-576-NR-2\injecting_numeracy\pre_training


Download the data files:

In [6]:
!bash download.sh

'bash' is not recognized as an internal or external command,
operable program or batch file.


Install the required libraries:

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 7.6MB 2.3MB/s 
     |████████████████████████████████| 296kB 33.4MB/s 
     |████████████████████████████████| 27.7MB 139kB/s 
     |████████████████████████████████| 655kB 41.0MB/s 
     |████████████████████████████████| 748.9MB 12kB/s 
     |████████████████████████████████| 1.5MB 41.5MB/s 
     |████████████████████████████████| 133kB 49.0MB/s 
     |████████████████████████████████| 163kB 48.5MB/s 
     |████████████████████████████████| 317kB 47.4MB/s 
     |████████████████████████████████| 184kB 39.5MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 5.3MB 40.3MB/s 
     |████████████████████████████████| 245kB 34.4MB/s 
     |████████████████████████████████| 133kB 49.7MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 266kB 47.7MB/s 
     |████████████████████████████████| 3.2MB 40.9MB/s 

# **Data Generation for pre-training/ finetuning on BERT** 
*Note:* Below 3 steps take a lot of time. So skip these three steps as synthetic data is already available in ./data folder  

**Step 1 - Masked LM Task (MLM) :** Tokenize and mask Wiki pages

In [ ]:
!python gen_train_data_MLM.py --train_corpus ./data/MLM_paras.jsonl --bert_model bert-base-uncased --output_dir ./data/MLM_train/ --do_lower_case --max_predictions_per_seq 65 --digitize

100% 231508/231508 [00:00<00:00, 4443190.86B/s]
384513it [00:18, 20619.17it/s]
 85% 325230/384513 [42:04<07:27, 132.40it/s]^C


**Step 2 - Numeric Data (ND) :** Generate numeric data and convert to DROP format

In [ ]:
%cd numeric_data_generation/       # navigate to numeric_data_generation folder

!python gen_numeric_data.py --num_samples 1e6 --num_dev_samples 1e4 --output_jsonl ../data/synthetic_numeric.jsonl  # generate numeric data

%cd ..                            # navigate back to pre-training folder

!python convert_synthetic_numeric_to_drop.py --data_jsonl ./data/synthetic_numeric.jsonl # convert numeric data to DROP format


[Errno 2] No such file or directory: 'numeric_data_generation/ # navigate to numeric_data_generation folder'
/content/injecting_numeracy/pre_training
python3: can't open file 'gen_numeric_data.py': [Errno 2] No such file or directory
[Errno 2] No such file or directory: '.. # navigate back to pre-training folder'
/content/injecting_numeracy/pre_training
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:__main__:Reading ./data/synthetic_numeric.jsonl
INFO:__main__:Converting...
INFO:__main__:Saving ./data/synthetic_numeric_train_drop_format.json
INFO:__main__:Saving ./data/synthetic_numeric_dev_drop_format.json


**Step 3 - Textual Data (TD) :** Convert textual data to DROP format.

In [ ]:
!python convert_synthetic_texual_to_drop.py --data_json ./data/synthetic_textual_mixed_min3_max6_up0.7_train.json
!python convert_synthetic_texual_to_drop.py --data_json ./data/synthetic_textual_mixed_min3_max6_up0.7_dev.json

INFO:__main__:Reading ./data/synthetic_textual_mixed_min3_max6_up0.7_train.json
tcmalloc: large alloc 3945979904 bytes == 0x78608000 @  0x7fb6363b21e7 0x5ac31b 0x7fb6341d3704 0x7fb6341d3417 0x7fb6341d4351 0x7fb6341d13fa 0x7fb6341d1554 0x50a4a5 0x50beb4 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x50ad03 0x634e72 0x634f27 0x6386df 0x639281 0x4b0dc0 0x7fb635fafbf7 0x5b259a
INFO:__main__:Converting...
INFO:__main__:Saving ./data/synthetic_textual_mixed_min3_max6_up0.7_train_drop_format.json
INFO:__main__:Reading ./data/synthetic_textual_mixed_min3_max6_up0.7_dev.json
INFO:__main__:Converting...
INFO:__main__:Saving ./data/synthetic_textual_mixed_min3_max6_up0.7_dev_drop_format.json


# **Pre-training / Finetuning models on DROP-like datasets**

In [ ]:
%cd gen_bert/ # navigate to gen_bert folder

[Errno 2] No such file or directory: 'gen_bert/ # navigate to gen_bert folder'
/content/injecting_numeracy/pre_training


**Step 1:** Creating features from DROP-like datasets

In [ ]:
# DROP
!python create_examples_n_features.py --split train --drop_json ../data/drop_dataset_train.json --output_dir data/examples_n_features --max_seq_length 512
!python create_examples_n_features.py --split eval --drop_json ../data/drop_dataset_dev.json --output_dir data/examples_n_features --max_seq_length 512

python3: can't open file 'create_examples_n_features.py': [Errno 2] No such file or directory
python3: can't open file 'create_examples_n_features.py': [Errno 2] No such file or directory


In [ ]:
# numeric data
!python create_examples_n_features.py --split train --drop_json ../data/synthetic_numeric_train_drop_format.json --output_dir data/examples_n_features_numeric --max_seq_length 50 --max_decoding_steps 11 --max_n_samples -1
!python create_examples_n_features.py --split eval --drop_json ../data/synthetic_numeric_dev_drop_format.json --output_dir data/examples_n_features_numeric --max_seq_length 50 --max_decoding_steps 11

# textual synthetic data
!python create_examples_n_features.py --split train --drop_json ../data/synthetic_textual_mixed_min3_max6_up0.7_train_drop_format.json --output_dir data/examples_n_features_syntext --max_seq_length 160 --max_n_samples -1
!python create_examples_n_features.py --split eval --drop_json ../data/synthetic_textual_mixed_min3_max6_up0.7_dev_drop_format.json --output_dir data/examples_n_features_syntext --max_seq_length 160

**Step 2:** Pre-training and finetuning

**PRE-TRAINING:**

GenBERT + ND

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 python finetune_on_drop.py  --do_train   --do_eval  --mlm_dir ../data/MLM_train/ --examples_n_features_dir ./data/examples_n_features_numeric/ --train_batch_size 800 --mlm_batch_size 48 --mlm_scale 0.5 --eval_batch_size 1200 --learning_rate 6e-5  --max_seq_length 50 --num_train_epochs 60.0 --warmup_proportion 0.1 --output_dir out_numeric_finetune_bert --random_shift --num_train_samples -1

python3: can't open file 'finetune_on_drop.py': [Errno 2] No such file or directory


GenBERT + TD

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 python finetune_on_drop.py   --do_train   --do_eval  --mlm_dir ../data/MLM_train/ --examples_n_features_dir ./data/examples_n_features_syntext/ --train_batch_size 240 --mlm_batch_size 48 --mlm_scale 0.5 --eval_batch_size 1000 --learning_rate 1e-5  --max_seq_length 160 --num_train_epochs 5.0 --warmup_proportion 0.1 --output_dir out_syntext_finetune_bert --random_shift --num_train_samples -1

GenBERT + ND + TD

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 python train_textual_with_numeric.py  --do_train   --do_eval --mlm_dir ../data/MLM_train/ --examples_n_features_dir_syntext ./data/examples_n_features_syntext/ --examples_n_features_dir_numeric ./data/examples_n_features_numeric/ --train_batch_size_syntext 240 --train_batch_size_numeric 624 --mlm_batch_size 48 --mlm_scale 0.5 --eval_batch_size 1000 --learning_rate 1e-5 --num_train_epochs 5.0 --warmup_proportion 0.1 --output_dir out_syntext_and_numeric_finetune_numeric --init_weights_dir out_numeric_finetune_bert --random_shift --num_train_samples -1

**FINETUNING:**

GenBERT + DROP

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1,2,3 python finetune_on_drop.py   --do_train   --do_eval  --examples_n_features_dir ./data/examples_n_features/ --train_batch_size 16 --mlm_batch_size -1 --eval_batch_size 360 --learning_rate 3e-5  --max_seq_length 512 --num_train_epochs 30.0 --warmup_proportion 0.1 --output_dir out_drop_finetune_bert --num_train_samples -1

GenBERT + ND + DROP

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1,2,3 python finetune_on_drop.py   --do_train   --do_eval  --examples_n_features_dir ./data/examples_n_features/ --train_batch_size 16 --mlm_batch_size -1 --eval_batch_size 360 --learning_rate 3e-5  --max_seq_length 512 --num_train_epochs 30.0 --warmup_proportion 0.1 --init_weights_dir out_numeric_finetune_bert --output_dir out_drop_finetune_numeric --num_train_samples -1

GenBERT + TD + DROP

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1,2,3 python finetune_on_drop.py   --do_train   --do_eval  --examples_n_features_dir ./data/examples_n_features/ --train_batch_size 14 --mlm_batch_size -1 --eval_batch_size 400 --learning_rate 3e-5  --max_seq_length 512 --num_train_epochs 30.0 --warmup_proportion 0.1 --init_weights_dir out_syntext_finetune_bert --output_dir out_drop_finetune_syntext --num_train_samples -1

GenBERT + ND + TD + DROP

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1,2,3 python finetune_on_drop.py   --do_train   --do_eval  --examples_n_features_dir ./data/examples_n_features/ --train_batch_size 14 --mlm_batch_size -1 --eval_batch_size 400 --learning_rate 3e-5  --max_seq_length 512 --num_train_epochs 30.0 --warmup_proportion 0.1 --init_weights_dir out_syntext_and_numeric_finetune_numeric --output_dir out_drop_finetune_syntext_and_numeric --num_train_samples -1

**Step 3:** Inference

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1,2,3 python finetune_on_drop.py --do_eval --do_inference --examples_n_features_dir ./data/examples_n_features/ --eval_batch_size 800 --init_weights_dir out_drop_finetuned_model  --output_dir preds

**Step 4:** Evaluation

In [ ]:
!python drop_eval.py --gold_path ../data/drop_dataset_dev.json --prediction_path ./preds/predictions.jsonl --answer_key prediction

**Step 5:** SQuAD Finetuning

In [ ]:
%cd squad_finetuning

!bash download.sh

In [ ]:
# training
!CUDA_VISIBLE_DEVICES=0 python run_squad.py  --bert_model bert-base-uncased  --do_train  --do_predict --do_lower_case --train_file data/train-v1.1.json   --predict_file data/dev-v1.1.json   --train_batch_size 13   --predict_batch_size 256   --learning_rate 3e-5   --num_train_epochs 3.0   --max_seq_length 384   --doc_stride 128   --overwrite_output_dir   --output_dir out_squad_bert_orig   --n_best_size 10   --eval_period 800  --load_from_cache --use_segment_ids --num_train_samples -1

In [ ]:
# inference
!CUDA_VISIBLE_DEVICES=0 python run_squad.py  --bert_model out_squad_bert_orig --do_evaluate --do_predict --do_lower_case --predict_file data/dev-v1.1.json  --predict_batch_size 256   --max_seq_length 384   --doc_stride 128  --use_segment_ids  --output_dir out_squad_bert_orig  --n_best_size 10

In [ ]:
# training
!CUDA_VISIBLE_DEVICES=0 python run_squad.py  --bert_model ../out_pre_training  --do_train  --do_predict --do_lower_case --train_file data/train-v1.1.json   --predict_file data/dev-v1.1.json   --train_batch_size 13   --predict_batch_size 256   --learning_rate 3e-5   --num_train_epochs 3.0   --max_seq_length 384   --doc_stride 128   --overwrite_output_dir  --indiv_digits  --output_dir out_squad_pre_training  --n_best_size 10   --eval_period 800  --load_from_cache --num_train_samples -1

In [ ]:
# inference
!CUDA_VISIBLE_DEVICES=0 python run_squad.py  --bert_model out_squad_pre_training --do_evaluate --do_predict --do_lower_case --predict_file data/dev-v1.1.json  --predict_batch_size 256   --max_seq_length 384   --doc_stride 128  --indiv_digits  --output_dir out_squad_pre_training   --n_best_size 10


In [ ]:
# evaluation
!python evaluate-v1.1.py --dataset_file data/dev-v1.1.json --prediction_file out_squad_pre_training/predictions.json